In [1]:
"""
In this notebook, I took the sample from pytorch documentation.
I added logging for all steps.
At the end, i print both dimensions and data.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random 

torch.manual_seed(1)

In [2]:
"""
A utility class to log all the sub steps of a
pytorch calculation.
Usage is sending data with label.
If data send is torch tensor, 
"""
class StepLogger():
    def __init__(self,capacity):
        self.tensor_datas = {}        
        self.capacity = capacity
        self.added_labels = []
        
    
    def add_info(self,tensor_data,tensor_label):
        if tensor_label not in self.added_labels:
            self.added_labels.append( tensor_label )
        
        if tensor_label in self.tensor_datas.keys():
            current_arr = self.tensor_datas.get(tensor_label)
            if len(current_arr) < self.capacity:
                current_arr = self.tensor_datas.get(tensor_label, [])
                current_arr.append(tensor_data)
        else:
            self.tensor_datas[tensor_label] = [tensor_data]
    
    def get_default_summary(self,show_data=False,summary_count=1):
        self.get_summary(self.added_labels,show_data)
        
    def get_summary(self,labels,show_data=False,summary_count=1):
        count = 0
        for i in range(summary_count):
            for l in labels:
                label_data = self.tensor_datas.get(l)[count]
                print(l," :")
                if torch.is_tensor(label_data):
                    print( list(label_data.size() ) )
                if not show_data and not torch.is_tensor(label_data):
                    print(label_data)
                if show_data:    
                    print(label_data)

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 7
HIDDEN_DIM = 8

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        step_logger.add_info(sentence,"forward sentence")
        
        embeds = self.word_embeddings(sentence)
        step_logger.add_info(embeds,"forward embeds")
        
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        step_logger.add_info(lstm_out,"forward lstm_out")
        
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        step_logger.add_info(tag_space,"forward tag_space")
        
        tag_scores = F.log_softmax(tag_space, dim=1)
        step_logger.add_info(tag_scores,"forward tag_scores")
        
        return tag_scores

In [5]:
step_logger =  StepLogger(2)


model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        step_logger.add_info(sentence,"train sentence")
        
        sentence_in = prepare_sequence(sentence, word_to_ix)
        step_logger.add_info(sentence_in,"train sentence_in")
        
        targets = prepare_sequence(tags, tag_to_ix)
        step_logger.add_info(targets,"train targets")

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        step_logger.add_info(tag_scores,"train tag_scores")
        step_logger.add_info(targets,"train targets")

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()



tensor([[-0.7253, -1.4652, -1.2560],
        [-0.7562, -1.4105, -1.2500],
        [-0.7989, -1.3465, -1.2378],
        [-0.8292, -1.4118, -1.1398],
        [-0.8057, -1.3929, -1.1879]])


In [6]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-0.0195, -4.6641, -4.6179],
        [-4.8760, -0.0134, -5.1667],
        [-4.0334, -3.8045, -0.0408],
        [-0.0260, -4.4073, -4.3085],
        [-4.2613, -0.0232, -4.7254]])


In [7]:
step_logger.get_default_summary(False)

forward sentence  :
[5]
forward embeds  :
[5, 7]
forward lstm_out  :
[5, 1, 8]
forward tag_space  :
[5, 3]
forward tag_scores  :
[5, 3]
train sentence  :
['The', 'dog', 'ate', 'the', 'apple']
train sentence_in  :
[5]
train targets  :
[5]
train tag_scores  :
[5, 3]


In [8]:
step_logger.get_default_summary(True)

forward sentence  :
[5]
tensor([0, 1, 2, 3, 4])
forward embeds  :
[5, 7]
tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798],
        [-1.6091, -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871],
        [ 0.2284,  0.4676, -0.6970, -1.1608,  0.6995,  0.1991,  0.8657],
        [ 0.2444, -0.6629,  0.8073,  1.1017, -0.1759, -2.2456, -1.4465],
        [ 0.0612, -0.6177, -0.7981, -0.1316,  1.8793, -0.0721,  0.1578]])
forward lstm_out  :
[5, 1, 8]
tensor([[[-0.0419, -0.0577, -0.0328,  0.0022, -0.0187, -0.1390,  0.1471,
           0.1007]],

        [[ 0.1646, -0.0772,  0.0060,  0.0156, -0.1495,  0.0406,  0.0428,
           0.0386]],

        [[ 0.0377,  0.0673,  0.0400, -0.0369, -0.1993, -0.0185,  0.0626,
          -0.1663]],

        [[ 0.0278, -0.0318, -0.0582,  0.2148, -0.1254,  0.1479,  0.0165,
          -0.0046]],

        [[-0.0126,  0.1178,  0.0768,  0.1926, -0.3055,  0.0439,  0.1439,
          -0.1545]]])
forward tag_space  :
[5, 3]
tensor([[ 0.3876, -0.3523, -